# Day 9: Earnings Trading Strategies

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week2_position_trend/day09_earnings_trading.ipynb)

---

## Learning Objectives

1. **Understand** how earnings announcements affect stock prices
2. **Identify** pre-earnings and post-earnings trading setups
3. **Analyze** earnings expectations and surprises
4. **Apply** gap trading strategies after earnings
5. **Manage** the elevated risk of earnings volatility

---

## Lecture (30 minutes)

### What Are Earnings?

**Earnings announcements** are quarterly reports where companies disclose financial performance. They are among the most volatile events for individual stocks.

```
EARNINGS REPORT COMPONENTS
==========================

KEY METRICS:
- EPS (Earnings Per Share)     - Net income / shares outstanding
- Revenue                      - Total sales
- Guidance                     - Company's future outlook
- Same-Store Sales             - Retail comparison
- User Metrics                 - Tech companies

WHAT MOVES STOCKS:
1. EPS Beat/Miss vs Consensus
2. Revenue Beat/Miss
3. Forward Guidance (most important!)
4. Management Commentary
5. Sector/Macro Implications
```

### Earnings Calendar

```
EARNINGS TIMING
===============

ANNOUNCEMENT TIMES:
- BMO (Before Market Open): 7:00-9:00 AM ET
- AMC (After Market Close): 4:00-5:00 PM ET
- During Market: Rare, usually special situations

EARNINGS SEASON:
- Jan-Feb: Q4 reports (most important)
- Apr-May: Q1 reports
- Jul-Aug: Q2 reports
- Oct-Nov: Q3 reports

SEQUENCE:
Banks/Financials → Tech → Consumer → Industrials → Retail
```

### Pre-Earnings Strategies

```
TRADING BEFORE EARNINGS
=======================

VOLATILITY RUN-UP:
- Implied volatility rises into earnings
- Stocks often drift toward expected move
- Options become expensive (IV crush after)

PRE-EARNINGS SETUPS:

1. MOMENTUM INTO EARNINGS
   - Stock trending up into report
   - Positive analyst sentiment
   - Sector strength
   - Exit BEFORE announcement

2. EARNINGS DRIFT
   - Stocks drift toward expected move
   - Historical pattern analysis
   - Follow institutional positioning

WARNING: Holding through earnings = gambling
Better to capture run-up, exit before report
```

### Post-Earnings Strategies

```
TRADING AFTER EARNINGS
======================

GAP TYPES:

1. GAP AND GO
   - Strong gap with follow-through
   - Volume confirming direction
   - Trade in gap direction
   
   |     /
   | GAP/    <- Entry on pullback
   |    
   +---+---

2. GAP AND FADE
   - Gap into resistance/support
   - Exhaustion pattern
   - Fade the gap
   
   |\    <- Resistance
   | \GAP
   |  \___
   +------

3. GAP FILL
   - Overreaction gap
   - Price returns to fill gap
   - Wait for confirmation
```

### Earnings Surprise Analysis

```
MEASURING SURPRISES
===================

EPS SURPRISE:
Surprise % = (Actual EPS - Estimate) / |Estimate| * 100

REACTION PATTERNS:

Beat + Raise Guidance  → Strong Up (Gap & Go)
Beat + Lower Guidance  → Mixed (May Fade)
Miss + Raise Guidance  → Mixed (May Recover)
Miss + Lower Guidance  → Strong Down

WHISPER NUMBERS:
- Unofficial expectations often higher than consensus
- "Beat" consensus but miss whisper = down
- Market may have already priced in beat
```

### Risk Management

```
EARNINGS RISK RULES
===================

POSITION SIZING:
- Reduce size 50-75% before earnings
- Or exit entirely before report
- Post-earnings: normal size after dust settles

STOP PLACEMENT:
- Pre-earnings: Tight stops, exit if trend breaks
- Post-earnings gap: Stop below gap low (longs)
- Give room for volatility expansion

HOLDING THROUGH EARNINGS:
- Binary outcome (up or down big)
- Stops don't protect in gaps
- Only for long-term investors
- Traders: capture run-up, exit before

EXPECTED MOVE:
- Options market implies expected move
- ATM straddle price ≈ expected move
- Moves beyond expected = opportunity
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
print("Libraries loaded!")

In [ ]:
def fetch_data(ticker, period='2y'):
    """Fetch historical data."""
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    df.index = pd.to_datetime(df.index)
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)
    return df

ticker = 'AAPL'
df = fetch_data(ticker)
print(f"Loaded {len(df)} days for {ticker}")

In [ ]:
def detect_earnings_gaps(df, gap_threshold=0.03):
    """Detect potential earnings gaps (large overnight moves)."""
    df = df.copy()
    
    # Calculate overnight gap
    df['Prev_Close'] = df['Close'].shift(1)
    df['Gap'] = (df['Open'] - df['Prev_Close']) / df['Prev_Close']
    df['Gap_Pct'] = df['Gap'] * 100
    
    # Calculate day's range and volume
    df['Day_Range'] = (df['High'] - df['Low']) / df['Open'] * 100
    df['Vol_Avg'] = df['Volume'].rolling(20).mean()
    df['Rel_Vol'] = df['Volume'] / df['Vol_Avg']
    
    # Large gaps (potential earnings)
    df['Large_Gap'] = abs(df['Gap']) > gap_threshold
    
    # Gap follow-through (did gap hold?)
    df['Gap_Up'] = df['Gap'] > gap_threshold
    df['Gap_Down'] = df['Gap'] < -gap_threshold
    
    # For gap ups: did close above open?
    df['Gap_Up_Hold'] = df['Gap_Up'] & (df['Close'] > df['Open'])
    df['Gap_Down_Hold'] = df['Gap_Down'] & (df['Close'] < df['Open'])
    
    return df

df_gaps = detect_earnings_gaps(df)

# Find significant gaps
large_gaps = df_gaps[df_gaps['Large_Gap']].copy()

print(f"\n{'='*60}")
print(f"EARNINGS GAP ANALYSIS: {ticker}")
print(f"{'='*60}")
print(f"\nTotal large gaps (>3%): {len(large_gaps)}")
print(f"Gap Ups: {len(large_gaps[large_gaps['Gap_Up']])}")
print(f"Gap Downs: {len(large_gaps[large_gaps['Gap_Down']])}")

if len(large_gaps) > 0:
    print(f"\nRecent Large Gaps (Potential Earnings):")
    print("-" * 60)
    for idx in large_gaps.tail(5).index:
        row = large_gaps.loc[idx]
        direction = "UP" if row['Gap'] > 0 else "DOWN"
        held = "HELD" if (row['Gap_Up_Hold'] or row['Gap_Down_Hold']) else "FADED"
        print(f"  {idx.strftime('%Y-%m-%d')}: {row['Gap_Pct']:+.1f}% {direction} ({held})")
        print(f"    Open: ${row['Open']:.2f} | Close: ${row['Close']:.2f} | Vol: {row['Rel_Vol']:.1f}x")

In [ ]:
def analyze_gap_performance(df, gap_threshold=0.03):
    """Analyze historical gap trading performance."""
    df = detect_earnings_gaps(df, gap_threshold)
    
    results = {
        'Gap Up': {'total': 0, 'held': 0, 'faded': 0, 'avg_return': []},
        'Gap Down': {'total': 0, 'held': 0, 'faded': 0, 'avg_return': []}
    }
    
    for idx in df.index:
        row = df.loc[idx]
        if pd.isna(row['Gap']):
            continue
            
        # Day return (open to close)
        day_return = (row['Close'] - row['Open']) / row['Open'] * 100
        
        if row['Gap_Up']:
            results['Gap Up']['total'] += 1
            results['Gap Up']['avg_return'].append(day_return)
            if row['Close'] > row['Open']:
                results['Gap Up']['held'] += 1
            else:
                results['Gap Up']['faded'] += 1
                
        elif row['Gap_Down']:
            results['Gap Down']['total'] += 1
            results['Gap Down']['avg_return'].append(day_return)
            if row['Close'] < row['Open']:
                results['Gap Down']['held'] += 1
            else:
                results['Gap Down']['faded'] += 1
    
    return results

gap_stats = analyze_gap_performance(df)

print(f"\n{'='*60}")
print("GAP TRADING STATISTICS")
print(f"{'='*60}\n")

for gap_type, stats in gap_stats.items():
    if stats['total'] > 0:
        hold_rate = stats['held'] / stats['total'] * 100
        avg_ret = np.mean(stats['avg_return']) if stats['avg_return'] else 0
        
        print(f"{gap_type}:")
        print(f"  Total Occurrences: {stats['total']}")
        print(f"  Held Direction: {stats['held']} ({hold_rate:.0f}%)")
        print(f"  Faded: {stats['faded']} ({100-hold_rate:.0f}%)")
        print(f"  Avg Day Return: {avg_ret:+.2f}%")
        print()

In [ ]:
def plot_earnings_gap(df, ticker, gap_date, days_before=5, days_after=10):
    """Plot price action around an earnings gap."""
    df = detect_earnings_gaps(df)
    
    # Get date range
    gap_idx = df.index.get_loc(gap_date)
    start_idx = max(0, gap_idx - days_before)
    end_idx = min(len(df), gap_idx + days_after + 1)
    
    window = df.iloc[start_idx:end_idx]
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), height_ratios=[3, 1])
    
    dates = range(len(window))
    gap_pos = gap_idx - start_idx
    
    # Price chart
    ax1 = axes[0]
    
    # Candlestick-style
    for i, (idx, row) in enumerate(window.iterrows()):
        color = 'green' if row['Close'] >= row['Open'] else 'red'
        ax1.plot([i, i], [row['Low'], row['High']], color=color, lw=1)
        ax1.plot([i, i], [row['Open'], row['Close']], color=color, lw=4)
    
    # Mark gap
    gap_row = window.iloc[gap_pos]
    ax1.axvline(x=gap_pos, color='blue', linestyle='--', alpha=0.5, label='Gap Day')
    
    # Show gap zone
    prev_close = window.iloc[gap_pos-1]['Close'] if gap_pos > 0 else gap_row['Open']
    gap_open = gap_row['Open']
    ax1.axhspan(min(prev_close, gap_open), max(prev_close, gap_open), 
                alpha=0.2, color='yellow', label='Gap Zone')
    
    ax1.set_title(f'{ticker} - Earnings Gap on {gap_date.strftime("%Y-%m-%d")}')
    ax1.set_ylabel('Price ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Volume
    ax2 = axes[1]
    colors = ['green' if window.iloc[i]['Close'] >= window.iloc[i]['Open'] else 'red' 
              for i in range(len(window))]
    ax2.bar(dates, window['Volume'] / 1e6, color=colors, alpha=0.7)
    ax2.axvline(x=gap_pos, color='blue', linestyle='--', alpha=0.5)
    ax2.set_ylabel('Volume (M)')
    ax2.set_xlabel('Days')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print gap details
    gap_pct = (gap_row['Open'] - prev_close) / prev_close * 100
    day_return = (gap_row['Close'] - gap_row['Open']) / gap_row['Open'] * 100
    
    print(f"\nGap Details:")
    print(f"  Gap Size: {gap_pct:+.1f}%")
    print(f"  Day Return (O→C): {day_return:+.1f}%")
    print(f"  Result: {'Gap Held' if (gap_pct > 0 and day_return > 0) or (gap_pct < 0 and day_return < 0) else 'Gap Faded'}")

# Plot most recent large gap
if len(large_gaps) > 0:
    recent_gap_date = large_gaps.index[-1]
    plot_earnings_gap(df, ticker, recent_gap_date)

In [ ]:
def earnings_gap_strategy(df, gap_threshold=0.03, hold_days=5):
    """Backtest simple post-earnings gap strategy."""
    df = detect_earnings_gaps(df, gap_threshold)
    
    trades = []
    
    for i in range(len(df) - hold_days):
        row = df.iloc[i]
        
        # Gap up: buy on first pullback (close < open) then hold
        if row['Gap_Up'] and row['Close'] >= row['Open']:  # Gap held first day
            entry = row['Close']
            exit_price = df.iloc[i + hold_days]['Close']
            return_pct = (exit_price - entry) / entry * 100
            
            trades.append({
                'Date': df.index[i],
                'Type': 'Gap Up Hold',
                'Entry': entry,
                'Exit': exit_price,
                'Return %': return_pct
            })
        
        # Gap down fade: buy gap down that shows reversal
        elif row['Gap_Down'] and row['Close'] > row['Open']:  # Gap faded (reversal)
            entry = row['Close']
            exit_price = df.iloc[i + hold_days]['Close']
            return_pct = (exit_price - entry) / entry * 100
            
            trades.append({
                'Date': df.index[i],
                'Type': 'Gap Down Fade',
                'Entry': entry,
                'Exit': exit_price,
                'Return %': return_pct
            })
    
    return pd.DataFrame(trades)

trades_df = earnings_gap_strategy(df)

print(f"\n{'='*60}")
print(f"POST-EARNINGS GAP STRATEGY BACKTEST")
print(f"{'='*60}\n")

if len(trades_df) > 0:
    print(f"Total Trades: {len(trades_df)}")
    print(f"Winners: {len(trades_df[trades_df['Return %'] > 0])}")
    print(f"Losers: {len(trades_df[trades_df['Return %'] <= 0])}")
    print(f"Win Rate: {len(trades_df[trades_df['Return %'] > 0]) / len(trades_df) * 100:.1f}%")
    print(f"Avg Return: {trades_df['Return %'].mean():+.2f}%")
    print(f"Best Trade: {trades_df['Return %'].max():+.2f}%")
    print(f"Worst Trade: {trades_df['Return %'].min():+.2f}%")
    
    print(f"\n\nRecent Trades:")
    print(trades_df.tail(5).to_string(index=False))
else:
    print("No trades generated with current parameters.")

---

## Quiz

In [ ]:
quiz = [
    {"q": "What typically moves a stock more than EPS beat/miss?", "a": "C", 
     "opts": ["A) Revenue beat", "B) Volume spike", "C) Forward guidance", "D) Analyst ratings"]},
    {"q": "When is it safest for traders to hold through earnings?", "a": "D",
     "opts": ["A) Always safe with stops", "B) When stock is trending up", 
              "C) When analysts expect a beat", "D) Never - it's binary risk"]},
    {"q": "What is a 'gap and go' pattern?", "a": "A",
     "opts": ["A) Gap with follow-through in gap direction", "B) Gap that fills immediately", 
              "C) Gap that reverses by close", "D) Small gap with no volume"]},
    {"q": "What does BMO mean in earnings context?", "a": "B",
     "opts": ["A) Buy More Options", "B) Before Market Open", 
              "C) Bearish Market Outlook", "D) Best Market Order"]},
    {"q": "How should position size change before earnings?", "a": "A",
     "opts": ["A) Reduce 50-75% or exit", "B) Double the position", 
              "C) Keep same size", "D) Only add if bullish"]}
]

def run_quiz():
    score = 0
    for i, q in enumerate(quiz):
        print(f"\nQ{i+1}: {q['q']}")
        for opt in q['opts']:
            print(f"   {opt}")
    print("\n" + "="*50)
    print("ANSWERS: 1-C, 2-D, 3-A, 4-B, 5-A")
    
run_quiz()

---

## Summary

- **Earnings** are high-volatility events that create trading opportunities
- **Pre-earnings**: Capture run-up, exit before report
- **Post-earnings**: Trade gap patterns (hold, fade, fill)
- **Guidance** matters more than beat/miss
- **Risk management**: Reduce size or exit before earnings

**Tomorrow**: Day 10 - Week 2 Review